# Capstone Project - Neighborhoods in Toronto

## PART 1
### Scraping data from Wikipedia

##### Import Libraries and get Web source data

In [1]:
# Install lxml parser
!pip install lxml
!pip install beautifulsoup4
!pip install requests

print("Libraries installed..")

#Import libraries
import requests
import json
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs
from pandas import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

You should consider upgrading via the 'c:\users\bobby mohanty\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.

You should consider upgrading via the 'c:\users\bobby mohanty\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.

You should consider upgrading via the 'c:\users\bobby mohanty\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.
Libraries installed..


In [2]:
# Load Data from Wikipedia
web_data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
print("Web data loaded..")

Web data loaded..


##### Scraping web source data 

In [3]:
# initialise the beautiful soup object with 'lxml' parser
soup = bs(web_data.text, 'lxml')

#using soup object, iterate the .wikitable to get the data from the HTML page and store it into a list
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

In [4]:
df = pd.DataFrame(table_contents)
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


In [5]:
# Dataframe Grouped by Postcode and joined with ","
grpdf = df.groupby(['PostalCode','Borough'], as_index=False, sort=False).agg(','.join)
grpdf.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


In [6]:
print("Dataframe shape : ", grpdf.shape)

Dataframe shape :  (103, 3)


## PART 2

### Get Longitude and Latitude data for postal codes

##### Load and initialize geo-spatial dataset

In [7]:
# Retreive the geo-spatial data
geoData = 'https://cocl.us/Geospatial_data'
# Load the data into a dataFrame
geoSpatial = pd.read_csv(geoData)
# Rename 'Postal Code' column to 'PostalCode'
newdf = geoSpatial.rename(columns ={'Postal Code':'PostalCode'}) 
newdf.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [8]:
# Merge by column name and create a new dataframe
mergedf=pd.merge(grpdf, newdf, on='PostalCode')
mergedf.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


## PART 3

### Analysis on the neighbourhoods of Toronto 
##### Install and Import folium python module

In [9]:
!pip install folium
import folium
print('Folium imported')

You should consider upgrading via the 'c:\users\bobby mohanty\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.
Folium imported


### 1. Visualize the dataset as a folium map

In [10]:
# Create folium Map object
map_toronto = folium.Map(location=[43.651070, -79.347015], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(mergedf['Latitude'], mergedf['Longitude'], mergedf['Borough'], mergedf['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto) 
    
map_toronto

#### Add Foursquare access secrets

In [50]:
# Add Foursquare access secrets
CLIENT_ID = ''  # Foursquare ID
CLIENT_SECRET = ''  # Foursquare Secret
VERSION = '20180605'  # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 
CLIENT_SECRET:


#### Lets only consider the boroughs having _"Downtown Toronto"_ word in their names

In [16]:

toronto_data = mergedf[mergedf['Borough'].str.contains("Downtown Toronto") ].reset_index(drop=True)
toronto_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
4,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383


#### Let's get the longitude and latitude of Toronto

In [13]:
# install geopy
!pip install geopy

# import geopy Nominatim
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

You should consider upgrading via the 'c:\users\bobby mohanty\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


In [17]:
address = 'Toronto, CN'

geolocator = Nominatim(user_agent="ny_explorer")
toronto_location = geolocator.geocode(address)
toronto_latitude = toronto_location.latitude
toronto_longitude = toronto_location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(toronto_latitude, toronto_longitude))

The geograpical coordinate of Manhattan are 43.6425637, -79.38708718320467.


#### Map for the current dataset with only the boroughs having _"Toronto"_ in their names

In [18]:
# Create folium Map object
map_toronto = folium.Map(location=[toronto_latitude, toronto_longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_data['Latitude'],
                                           toronto_data['Longitude'],
                                           toronto_data['Borough'],
                                           toronto_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto) 
    
map_toronto

#### Let's explore the first neighborhood in our dataframe.

##### Get the neighborhood's name.

In [19]:
toronto_data.loc[0, 'Neighborhood']

'Regent Park, Harbourfront'

##### Get the neighborhood's latitude and longitude values.

In [20]:

neighborhood_latitude = toronto_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = toronto_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = toronto_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Regent Park, Harbourfront are 43.6542599, -79.3606359.


Now, let's get the top 100 venues that are in *"Regent Park, Harbourfront"* within a radius of 500 meters.

In [21]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=2GEAB5OS4EDLSI2P4HOBOI5R5HVZJOLHXMY1RZKC3UCDGJBL&client_secret=YFPVHLUPBSZVUY53FJCRH0WH43OZUQUDVYFZ1SI30IQLF3P2&v=20180605&ll=43.6542599,-79.3606359&radius=500&limit=100'

Send the GET request and examine the results

In [22]:
results = requests.get(url).json()
results

565116074695,
        'lng': -79.35784287026658,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.650565116074695,
          'lng': -79.35784287026658}],
        'distance': 468,
        'postalCode': 'M5A 3C4',
        'cc': 'CA',
        'neighborhood': 'Distillery District, Toronto, ON',
        'city': 'Toronto',
        'state': 'ON',
        'country': 'Canada',
        'formattedAddress': ['35 Tank House Lane',
         'Toronto ON M5A 3C4',
         'Canada']},
       'categories': [{'id': '4bf58dd8d48988d10c941735',
         'name': 'French Restaurant',
         'pluralName': 'French Restaurants',
         'shortName': 'French',
         'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/french_',
          'suffix': '.png'},
         'primary': True}],
       'photos': {'count': 0, 'groups': []},
       'venuePage': {'id': '199972479'}},
      'referralId': 'e-0-53a22c92498ec91fda7ce133-25'},
     {'reasons': {'count': 0,
       'items': [{'sum

Function that extracts the category of the venue

In [23]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Clean the result and convert to pandas dataframe.

In [24]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Tandem Coffee,Coffee Shop,43.653559,-79.361809
1,Roselle Desserts,Bakery,43.653447,-79.362017
2,Cooper Koo Family YMCA,Distribution Center,43.653249,-79.358008
3,Body Blitz Spa East,Spa,43.654735,-79.359874
4,Impact Kitchen,Restaurant,43.656369,-79.356980


In [25]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

42 venues were returned by Foursquare.


### 2. Explore Neighborhoods in Toronto
#### Let's create a function to repeat the same process to all the neighborhoods in Toronto

In [26]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Let's run the above function for the neighborhoods of Toronto.

In [28]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
Enclave of M5E
St. James Town, Cabbagetown
First Canadian Place, Underground city
Church and Wellesley


#### Let's check the size of the resulting dataframe

In [29]:
print(toronto_venues.shape)
toronto_venues.head()

(1200, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
1,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Regent Park, Harbourfront",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant


Let's check how many venues were returned for each neighborhood

In [30]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,58,58,58,58,58,58
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",16,16,16,16,16,16
Central Bay Street,65,65,65,65,65,65
Christie,16,16,16,16,16,16
Church and Wellesley,79,79,79,79,79,79
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Enclave of M5E,99,99,99,99,99,99
"First Canadian Place, Underground city",100,100,100,100,100,100
"Garden District, Ryerson",100,100,100,100,100,100


#### Let's find out how many unique categories can be curated from all the returned venues

In [31]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 204 uniques categories.


### 3. Analyze Each Neighborhood

In [32]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Adult Boutique,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


let's examine the new dataframe size.

In [33]:
toronto_onehot.shape

(1200, 204)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [34]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Adult Boutique,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar
0,Berczy Park,0.000000,0.000000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,...,0.000000,0.017241,0.000000,0.000000,0.00,0.00,0.017241,0.000000,0.000000,0.000000
1,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.000000,0.0625,0.0625,0.125,0.125,0.125,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000
2,Central Bay Street,0.015385,0.000000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,...,0.015385,0.015385,0.000000,0.000000,0.00,0.00,0.015385,0.000000,0.000000,0.015385
3,Christie,0.000000,0.000000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000
4,Church and Wellesley,0.025316,0.012658,0.0000,0.0000,0.000,0.000,0.000,0.012658,0.000000,...,0.000000,0.012658,0.012658,0.012658,0.00,0.00,0.000000,0.000000,0.000000,0.000000
5,"Commerce Court, Victoria Hotel",0.000000,0.000000,0.0000,0.0000,0.000,0.000,0.000,0.020000,0.000000,...,0.010000,0.010000,0.000000,0.000000,0.00,0.00,0.020000,0.000000,0.000000,0.010000
6,Enclave of M5E,0.010101,0.000000,0.0000,0.0000,0.000,0.000,0.000,0.010101,0.010101,...,0.000000,0.010101,0.000000,0.000000,0.00,0.00,0.010101,0.000000,0.000000,0.000000
7,"First Canadian Place, Underground city",0.000000,0.000000,0.0000,0.0000,0.000,0.000,0.000,0.020000,0.000000,...,0.010000,0.000000,0.010000,0.000000,0.00,0.01,0.010000,0.000000,0.000000,0.010000
8,"Garden District, Ryerson",0.000000,0.000000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,...,0.010000,0.010000,0.020000,0.000000,0.00,0.00,0.000000,0.010000,0.010000,0.010000
9,"Harbourfront East, Union Station, Toronto Islands",0.000000,0.000000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,...,0.010000,0.010000,0.000000,0.000000,0.00,0.01,0.000000,0.000000,0.000000,0.010000


#### Let's confirm the new size

In [35]:
toronto_grouped.shape

(18, 204)

#### Let's print each neighborhood along with the top 5 most common venues

In [37]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                venue  freq
0         Coffee Shop  0.10
1        Cocktail Bar  0.05
2              Bakery  0.05
3  Seafood Restaurant  0.03
4          Restaurant  0.03


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
              venue  freq
0    Airport Lounge  0.12
1   Airport Service  0.12
2  Airport Terminal  0.12
3     Boat or Ferry  0.06
4             Plane  0.06


----Central Bay Street----
                 venue  freq
0          Coffee Shop  0.17
1       Sandwich Place  0.05
2                 Café  0.05
3   Italian Restaurant  0.05
4  Japanese Restaurant  0.03


----Christie----
           venue  freq
0  Grocery Store  0.25
1           Café  0.19
2           Park  0.12
3      Nightclub  0.06
4     Restaurant  0.06


----Church and Wellesley----
                 venue  freq
0          Coffee Shop  0.08
1     Sushi Restaurant  0.06
2  Japanese Restaurant  0.06
3              Gay Bar  0.04
4  

#### Let's put that into a _pandas_ dataframe

In [38]:
# function to sort the venues in descending order.
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

let's create the new dataframe and display the top 10 venues for each neighborhood.

In [39]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Bakery,Seafood Restaurant,Restaurant,Cheese Shop,Beer Bar,Farmers Market,Breakfast Spot,Belgian Restaurant
1,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Lounge,Airport Service,Airport Terminal,Boat or Ferry,Plane,Boutique,Sculpture Garden,Rental Car Location,Bar,Coffee Shop
2,Central Bay Street,Coffee Shop,Sandwich Place,Café,Italian Restaurant,Japanese Restaurant,Burger Joint,Salad Place,Restaurant,Bubble Tea Shop,Deli / Bodega
3,Christie,Grocery Store,Café,Park,Nightclub,Restaurant,Candy Store,Italian Restaurant,Baby Store,Athletics & Sports,Coffee Shop
4,Church and Wellesley,Coffee Shop,Sushi Restaurant,Japanese Restaurant,Gay Bar,Restaurant,Yoga Studio,Mediterranean Restaurant,Fast Food Restaurant,Café,Men's Store


### 4. Cluster Neighborhoods

Run _k_-means to cluster the neighborhood into 3 clusters.

In [40]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 3, 0, 2, 0, 0, 0, 0, 0, 0])

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [41]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_new_merged = toronto_data

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
toronto_new_merged = toronto_new_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_new_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0,Coffee Shop,Bakery,Pub,Park,Breakfast Spot,Theater,Café,Event Space,Chocolate Shop,Dessert Shop
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,0,Coffee Shop,Clothing Store,Sandwich Place,Hotel,Café,Middle Eastern Restaurant,Cosmetics Shop,Japanese Restaurant,Italian Restaurant,Pizza Place
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Coffee Shop,Café,Cocktail Bar,Beer Bar,Gastropub,Moroccan Restaurant,Seafood Restaurant,Clothing Store,Restaurant,Creperie
3,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,0,Coffee Shop,Cocktail Bar,Bakery,Seafood Restaurant,Restaurant,Cheese Shop,Beer Bar,Farmers Market,Breakfast Spot,Belgian Restaurant
4,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,0,Coffee Shop,Sandwich Place,Café,Italian Restaurant,Japanese Restaurant,Burger Joint,Salad Place,Restaurant,Bubble Tea Shop,Deli / Bodega


In [42]:
toronto_new_merged.dropna(inplace=True)

Finally, let's visualize the resulting clusters

In [43]:
# create map
map_clusters = folium.Map(location=[toronto_latitude, toronto_longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_new_merged['Latitude'],
                                  toronto_new_merged['Longitude'],
                                  toronto_new_merged['Neighborhood'],
                                  toronto_new_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    cluster = int(cluster)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters


### 5. Examine Clusters
#### Cluster 1

In [44]:
toronto_new_merged.loc[toronto_new_merged['Cluster Labels'] == 0,
                       toronto_new_merged.columns[[1] + list(range(5, toronto_new_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,0,Coffee Shop,Bakery,Pub,Park,Breakfast Spot,Theater,Café,Event Space,Chocolate Shop,Dessert Shop
1,Downtown Toronto,0,Coffee Shop,Clothing Store,Sandwich Place,Hotel,Café,Middle Eastern Restaurant,Cosmetics Shop,Japanese Restaurant,Italian Restaurant,Pizza Place
2,Downtown Toronto,0,Coffee Shop,Café,Cocktail Bar,Beer Bar,Gastropub,Moroccan Restaurant,Seafood Restaurant,Clothing Store,Restaurant,Creperie
3,Downtown Toronto,0,Coffee Shop,Cocktail Bar,Bakery,Seafood Restaurant,Restaurant,Cheese Shop,Beer Bar,Farmers Market,Breakfast Spot,Belgian Restaurant
4,Downtown Toronto,0,Coffee Shop,Sandwich Place,Café,Italian Restaurant,Japanese Restaurant,Burger Joint,Salad Place,Restaurant,Bubble Tea Shop,Deli / Bodega
6,Downtown Toronto,0,Coffee Shop,Café,Thai Restaurant,Restaurant,Clothing Store,Deli / Bodega,Gym,Hotel,Seafood Restaurant,Sushi Restaurant
7,Downtown Toronto,0,Coffee Shop,Café,Pizza Place,Aquarium,Restaurant,Hotel,Scenic Lookout,Italian Restaurant,Sandwich Place,Steakhouse
8,Downtown Toronto,0,Coffee Shop,Hotel,Café,Restaurant,Japanese Restaurant,Bakery,Salad Place,Seafood Restaurant,Lounge,Breakfast Spot
9,Downtown Toronto,0,Coffee Shop,Café,Hotel,Restaurant,Cocktail Bar,Gym,Italian Restaurant,Japanese Restaurant,Deli / Bodega,Asian Restaurant
14,Downtown TorontoStn A PO Boxes25 The Esplanade,0,Coffee Shop,Italian Restaurant,Seafood Restaurant,Bakery,Café,Cocktail Bar,Restaurant,Japanese Restaurant,Pub,Hotel


#### Cluster 2

In [45]:
toronto_new_merged.loc[toronto_new_merged['Cluster Labels'] == 1,
                       toronto_new_merged.columns[[1] + list(range(5, toronto_new_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
13,Downtown Toronto,1,Park,Trail,Playground,Monument / Landmark,Market,Martial Arts School,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant


#### Cluster 3

In [47]:
toronto_new_merged.loc[toronto_new_merged['Cluster Labels'] == 2,
                       toronto_new_merged.columns[[1] + list(range(5, toronto_new_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Downtown Toronto,2,Grocery Store,Café,Park,Nightclub,Restaurant,Candy Store,Italian Restaurant,Baby Store,Athletics & Sports,Coffee Shop


#### Cluster 4

In [48]:
toronto_new_merged.loc[toronto_new_merged['Cluster Labels'] == 3,
                       toronto_new_merged.columns[[1] + list(range(5, toronto_new_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,Downtown Toronto,3,Airport Lounge,Airport Service,Airport Terminal,Boat or Ferry,Plane,Boutique,Sculpture Garden,Rental Car Location,Bar,Coffee Shop


#### Cluster 5

In [49]:
toronto_new_merged.loc[toronto_new_merged['Cluster Labels'] == 4,
                       toronto_new_merged.columns[[1] + list(range(5, toronto_new_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,Downtown Toronto,4,Café,Bakery,Japanese Restaurant,Bookstore,Bar,Yoga Studio,Poutine Place,Beer Store,Sandwich Place,Restaurant
11,Downtown Toronto,4,Café,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Bar,Bakery,Coffee Shop,Park,Mexican Restaurant,Burger Joint,Caribbean Restaurant
